Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [182]:
import random
from abc import abstractmethod
import numpy as np
import lab9_lib


In [183]:
LOCI_NUMBER = 1000
MAX_ITERATIONS = 1500

class Individual:
    def __init__(self, genotype=None):
        self.genotype = genotype if genotype else random.choices([0, 1], k=LOCI_NUMBER)

    @staticmethod
    def evaluate_population(population, fitness_fuction) -> dict['Individual', float]:
        return {ind: fitness_fuction(ind.genotype) for ind in population}
    
    @staticmethod
    def difference(ind1, ind2) -> float:
        return sum(np.logical_xor(ind1.genotype, ind2.genotype)) / LOCI_NUMBER if ind1 and ind2 else 1
    
    @staticmethod
    @abstractmethod
    def population(size) -> list['Individual']:
        pass

    @staticmethod
    @abstractmethod
    def algorithm() :
        pass

## Hill climbing

In [184]:
class HillIndividual(Individual):
    POPULATION_NUMBER = 1
    OFFSPRING_NUMBER = 10

    def __init__(self, genotype=None):
        super().__init__(genotype)

    def population() -> list['HillIndividual']:
        return [HillIndividual() for _ in range(HillIndividual.POPULATION_NUMBER)]

    def tweak(self, index=-1) -> 'Individual':
        chunks_number = random.choice([2 ** i for i in range(np.log2(LOCI_NUMBER).astype(int))])
        chunks = np.array_split(self.genotype, chunks_number)
        random.shuffle(chunks)
        genotype = np.concatenate(chunks)
        return HillIndividual(genotype.tolist())
    
    @staticmethod
    def algorithm(population, fitness_function, max_iterations=MAX_ITERATIONS) -> tuple[tuple['HillIndividual', float], int]:
        population = HillIndividual.evaluate_population(population, fitness_function)

        for i in range(max_iterations):
            new_population = {}
            for individual, evaluation in population.items():
                new_individuals = [individual.tweak() for _ in range(HillIndividual.OFFSPRING_NUMBER)] 
                evalueated_individuals = HillIndividual.evaluate_population(new_individuals, fitness_function)
                best_individual, best_evaluation = max(evalueated_individuals.items(), key=lambda x: x[1])
                print(f'{i}/{max_iterations} - {best_evaluation:.2%}: {''.join(str(g) for g in best_individual.genotype)}', end='\r')
                if best_evaluation > evaluation:
                    new_population[best_individual] = best_evaluation
                    if best_evaluation == 1:
                        return (best_individual, best_evaluation), fitness_function.calls
                else:
                    new_population[individual] = evaluation
            population = new_population
        
        return max(population.items(), key=lambda x: x[1]), fitness_function.calls
            
                
                

## Genetic algorithm

In [185]:

class GeneticIndividual(Individual):
    POPULATION_NUMBER = 100
    POPULATION_INCREASE = 0.01
    OFFSPRING_NUMBER = 10
    OFFSPRING_INCREASE = 0.001
    MUTATION_PROBABILITY = 0.4
    MUTATION_PROBABILITY_INCREASE = 0
    MUTATION_METHOD = 2
    CROSSOVER_METHOD = 0
    SELECTIVE_METHOD = 1

    def __init__(self, genotype=None):
        super().__init__(genotype)

    @staticmethod
    def single_mutation(ind) -> 'GeneticIndividual':
        genotype = ind.genotype.copy()
        index = random.choice(range(LOCI_NUMBER))
        genotype[index] = 1 - genotype[index]
        return GeneticIndividual(genotype)
    
    def multi_mutation(ind, probabilty=1/LOCI_NUMBER) -> 'GeneticIndividual':
        genotype = [1 - g if random.random() < probabilty else g for g in ind.genotype]
        return GeneticIndividual(genotype)
    
    def chunk_mutation(ind) -> 'GeneticIndividual':
        chunks_number = random.choice([2 ** i for i in range(np.log2(LOCI_NUMBER).astype(int))])
        chunks = np.array_split(ind.genotype, chunks_number)
        random.shuffle(chunks)
        genotype = np.concatenate(chunks)
        return GeneticIndividual(genotype.tolist())

    @staticmethod
    def scrumble_crossover(ind1, ind2) -> 'GeneticIndividual':
        genotype = [random.choice([g1, g2]) for g1, g2 in zip(ind1.genotype, ind2.genotype)]
        return GeneticIndividual(genotype)
    
    @staticmethod
    def cut_crossover(ind1, ind2) -> 'GeneticIndividual':
        index = random.choice(range(LOCI_NUMBER))
        genotype = ind1.genotype[:index] + ind2.genotype[index:]
        return GeneticIndividual(genotype)
    
    @staticmethod
    def chunck_crossover(ind1, ind2) -> 'GeneticIndividual':
        chunks_number = random.choice([2 ** i for i in range(np.log2(LOCI_NUMBER).astype(int))])
        chunks1 = np.array_split(ind1.genotype, chunks_number)
        chunks2 = np.array_split(ind2.genotype, chunks_number)
        genotype = np.concatenate([random.choice([chunk1, chunk2]) for chunk1, chunk2 in zip(chunks1, chunks2)])
        return GeneticIndividual(genotype.tolist())
    
    @staticmethod
    def roulette_selection(population, iteration=0) -> list['GeneticIndividual']:
        new_population = random.choices(list(population.keys()), weights=population.values(), k=round(GeneticIndividual.OFFSPRING_NUMBER + GeneticIndividual.OFFSPRING_INCREASE*iteration*iteration))
        return new_population

    @staticmethod
    def tournament_selection(population, tournament_size=2, iteration=0) -> list['GeneticIndividual']:
        new_population = []
        for _ in range(round(GeneticIndividual.OFFSPRING_NUMBER + GeneticIndividual.OFFSPRING_INCREASE*iteration*iteration)):
            tournament = random.choices(list(population.keys()), k=tournament_size)
            new_population.append(max(tournament, key=lambda x: population[x]))
        
        return new_population
    
    @staticmethod
    def difference_selection(population, iteration=0) -> list['GeneticIndividual']:
        new_population = []
        new_individual = None
        for _ in range(round(GeneticIndividual.OFFSPRING_NUMBER + GeneticIndividual.OFFSPRING_INCREASE*iteration*iteration)):
            scaled_population = {ind: value*(1 + Individual.difference(ind, new_individual)) for ind, value in population.items()}
            new = random.choices(list(scaled_population.keys()), weights=scaled_population.values(), k=1)[0]
            print(GeneticIndividual.difference(new, new_individual), end='\r')
            new_individual = new
            new_population.append(new_individual)
        
        return new_population
                
    @staticmethod
    def new_generation(population,iteration=0) -> list['GeneticIndividual']:
        new_population = []
        for _ in range(round(GeneticIndividual.POPULATION_NUMBER + GeneticIndividual.POPULATION_INCREASE*iteration*iteration)):
            if random.random() < (GeneticIndividual.MUTATION_PROBABILITY + GeneticIndividual.MUTATION_PROBABILITY_INCREASE*iteration) % 1:
                ind = random.choice(population)
                if GeneticIndividual.MUTATION_METHOD == 0:   
                    new_population.append(GeneticIndividual.single_mutation(ind)) 
                elif GeneticIndividual.MUTATION_METHOD == 1:
                    new_population.append(GeneticIndividual.multi_mutation(ind, (5*(iteration + 1)%LOCI_NUMBER)/LOCI_NUMBER))
                elif GeneticIndividual.MUTATION_METHOD == 2:
                    new_population.append(GeneticIndividual.chunk_mutation(ind))
                else:
                    if random.random() < 0.5:
                        new_population.append(GeneticIndividual.chunk_mutation(ind))
                    else:
                        new_population.append(GeneticIndividual.single_mutation(ind))
                        
            else:
                ind1, ind2 = random.choices(population, k=2)
                if GeneticIndividual.CROSSOVER_METHOD == 0:                    
                    new_population.append(GeneticIndividual.scrumble_crossover(ind1, ind2))
                elif GeneticIndividual.CROSSOVER_METHOD == 1:
                    new_population.append(GeneticIndividual.cut_crossover(ind1, ind2))
                else:
                    new_population.append(GeneticIndividual.chunck_crossover(ind1, ind2))
        return new_population

    @staticmethod
    def population() -> list['GeneticIndividual']:
        return [GeneticIndividual() for _ in range(GeneticIndividual.POPULATION_NUMBER)]
    
    @staticmethod
    def epoch(population, fitness_function, iteration=0) -> dict['GeneticIndividual', float]:
        if GeneticIndividual.SELECTIVE_METHOD == 0:
            offspring = GeneticIndividual.roulette_selection(population, iteration)
        elif GeneticIndividual.SELECTIVE_METHOD == 1:
            offspring = GeneticIndividual.tournament_selection(population, 100, iteration)
        else:
            offspring = GeneticIndividual.difference_selection(population, iteration)
        new_population = GeneticIndividual.new_generation(offspring, iteration)
        return GeneticIndividual.evaluate_population(new_population, fitness_function)

    @staticmethod
    def algorithm(population, fitness_function, max_iterations=MAX_ITERATIONS):
        population = GeneticIndividual.evaluate_population(population, fitness_function)
        best = (None, None)
        for i in range(max_iterations):
            population = GeneticIndividual.epoch(population, fitness_function, i)
            last = best
            best = max(population.items(), key=lambda x: x[1])
            print(f'{i}/{max_iterations} - {round(GeneticIndividual.difference(last[0], best[0])*LOCI_NUMBER)} - {best[1]:.2%}: {''.join(str(g) for g in best[0].genotype)}', end='\r')
            if max(population.values()) >= 1:
                break
        
        return max(population.items(), key=lambda x: x[1]), fitness_function.calls

### Genetic algorithm with isolation

In [186]:
class IsolationIndividual(GeneticIndividual):
    ISLAND_NUMBER = 5
    ISLAND_ITERATIONS = 10

    def __init__(self, genotype=None):
        super().__init__(genotype)

    @staticmethod
    def algorithm(population, fitness_function, max_iterations=MAX_ITERATIONS):
        population = GeneticIndividual.evaluate_population(population, fitness_function)

        islands = np.array_split(list(population.items()), IsolationIndividual.ISLAND_NUMBER)
        last = [None]*IsolationIndividual.ISLAND_NUMBER
        best = (None, None)
        for i in range(max_iterations):
            for j, island in enumerate(islands):
                last[j] = best
                best = max(island, key=lambda x: x[1])
                island = GeneticIndividual.epoch(dict(island), fitness_function).items()
                print(f'{i}/{max_iterations} - {j}/{IsolationIndividual.ISLAND_NUMBER} - {round(GeneticIndividual.difference(last[j][0], best[0])*LOCI_NUMBER)} - {best[1]:.2%}: {''.join(str(g) for g in best[0].genotype)}', end='\r')
            population = np.concatenate(islands)
           
            if max(population, key=lambda x: x[1])[1] == 1:
                break

            if (i+1) % IsolationIndividual.ISLAND_ITERATIONS == 0:               
                random.shuffle(population)
                islands = np.array_split(population, IsolationIndividual.ISLAND_NUMBER)

        return max(population.items(), key=lambda x: x[1]), fitness_function.calls        


## Algorithm

In [187]:
types = [HillIndividual, GeneticIndividual]

for problem in [1, 2, 5, 10]:
    print(f'\nProblem {problem}')
    for t in types:
        fitness = lab9_lib.make_problem(problem)
        
        best, calls = t.algorithm(t.population(), fitness)
        print(f'\t{t.__name__}' + ' '*1100)
        #print(f'\t\tBest individual: {''.join(str(g) for g in best[0].genotype)}')
        print(f'\t\tBest evaluation: {best[1]:.2%}')
        print(f'\t\tNumber of calls: {round(calls/1000)} K')
    



Problem 1
	HillIndividual                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              